# 8. Unsupervised Topic Modelling via LDA (Initial)

In [ ]:
from google.colab import drive

# Mounting Google Drive
drive.mount('/content/drive', force_remount=True) # Adding force_remount=True to force the remounting process

Mounted at /content/drive


In [ ]:
import pandas as pd
train_df = pd.read_csv('/content/drive/MyDrive/train_data.csv')
train_df

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review
0,46979,43342,52189,Andrew Ng has provided a fantastic resource fo...,2017-09-12,5,machine-learning,2017-09,Computer Science,Machine Learning,Stanford University,positive,admiration,5 stars,positive
1,15087,27963,33570,Peer graded assingments take a lot of time bec...,2020-06-14,4,python-databases,2020-06,Computer Science,Using Databases with Python,University of Michigan,positive,neutral,4 stars,positive
2,32265,3290,4397,This course was the first actual course for Pr...,2019-01-31,5,python,2019-01,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive
3,44070,43900,52762,The best course to start with if you are enter...,2017-08-20,5,machine-learning,2017-08,Computer Science,Machine Learning,Stanford University,positive,approval,5 stars,positive
4,18016,34850,42219,I wanted to learn more about Functional Progra...,2020-07-12,4,programming-languages,2020-07,Computer Science,"Programming Languages, Part A",University of Washington,positive,approval,5 stars,positive
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39733,14444,32000,38501,This course is one of the best courses that sh...,2019-02-12,4,python-data-visualization,2019-02,Computer Science,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,positive,admiration,5 stars,positive
39734,46433,25044,30104,"The details about Objects, References and Alia...",2019-08-08,4,python-basics,2019-08,Computer Science,Python Basics,University of Michigan,positive,admiration,4 stars,positive
39735,22140,27877,33439,This course gives very basic information about...,2016-02-19,4,python-databases,2016-02,Computer Science,Using Databases with Python,University of Michigan,positive,admiration,4 stars,positive
39736,29009,2544,3541,Awesome courseDr. Chuck is great,2019-02-03,4,python,2019-02,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,5 stars,positive


In [ ]:
train_df[train_df['rating'] == 1]

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,sentiment_label,sentiment_label_from_review
82,7432,17000,20049,Not met what offered. I really don't know why ...,2019-05-28,1,data-scientists-tools,2019-05,Computer Science,The Data Scientist’s Toolbox,Johns Hopkins University,negative,confusion,2 stars,negative
96,6718,29502,35306,"Very unsatisfied. At the end, I feel I learnt ...",2017-09-01,1,algorithmic-toolbox,2017-09,Computer Science,Algorithmic Toolbox,University of California San Diego,negative,disappointment,2 stars,negative
120,15597,49,58,"Better off using Udemy, more organized, cheape...",2020-01-02,1,python,2020-01,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,negative,disapproval,2 stars,negative
133,7172,31897,38278,Instructions for assignments not clear. TA not...,2019-02-18,1,python-data-visualization,2019-02,Computer Science,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,negative,disapproval,2 stars,negative
197,1558,17045,20104,"Certainly not worth paying for this course, on...",2016-03-20,1,data-scientists-tools,2016-03,Computer Science,The Data Scientist’s Toolbox,Johns Hopkins University,negative,neutral,2 stars,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39452,29501,31909,38293,I absolutely didn't get the purpose of this co...,2020-05-15,1,python-data-visualization,2020-05,Computer Science,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,negative,confusion,1 star,negative
39475,6912,38195,46268,Was shocked by Math! You have to be perfect at...,2018-01-14,1,machine-learning,2018-01,Computer Science,Machine Learning,Stanford University,negative,surprise,5 stars,positive
39520,42182,35897,43506,this is the worst course ever. so bored and fr...,2020-03-03,1,python-machine-learning,2020-03,Computer Science,Applied Machine Learning in Python,University of Michigan,negative,disgust,1 star,negative
39665,49073,36698,44479,Unable to download and install cloudera vm ins...,2017-08-24,1,big-data-introduction,2017-08,Computer Science,Introduction to Big Data,University of California San Diego,negative,disapproval,1 star,negative


In [ ]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

# Download NLTK stopwords and punkt
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))


df_reviews = train_df.head(100)

# Preprocessing function
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Tokenizing and converting to lowercase
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Function to get top topics for each LDA model
def get_topic_label(lda_model, corpus, num_topics):
    # Get the most dominant topic for each review
    topic_labels = []
    for doc in corpus:
        topics = lda_model.get_document_topics(doc)
        dominant_topic = max(topics, key=lambda x: x[1])[0]  # Get the topic with the highest probability
        topic_labels.append(f"Topic {dominant_topic + 1}")
    return topic_labels

# Function to get LDA topic summaries (top words for each topic)
def get_lda_topic_summary(lda_model, num_topics, top_n=5):
    summary = {}
    for i in range(num_topics):
        # Get the top words for each topic
        words = lda_model.show_topic(i, topn=top_n)
        topic_keywords = " + ".join([f'{weight:.3f}*"{word}"' for word, weight in words])
        summary[f"Topic {i+1}"] = topic_keywords
    return summary

# Function to run LDA with different topic numbers and add them as columns to the dataframe
def run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=100, checkpoint_dir="checkpoints"):
    all_topic_labels = {}
    all_topic_summaries = {}

    # Preprocess the reviews
    df_reviews['processed_reviews'] = df_reviews['reviews'].apply(preprocess)
    dictionary = corpora.Dictionary(df_reviews['processed_reviews'])
    corpus = [dictionary.doc2bow(review) for review in df_reviews['processed_reviews']]

    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Loop through different topic numbers (from topic_range)
    for num_topics in range(topic_range[0], topic_range[1] + 1):
        batch_num = 0
        topic_labels_list = []

        # Check if checkpoint exists for this number of topics
        checkpoint_file = os.path.join(checkpoint_dir, f"lda_model_{num_topics}_topics.csv")

        if os.path.exists(checkpoint_file):
            print(f"Skipping LDA with {num_topics} topics, checkpoint found.")
            continue

        # Batch processing
        for i in range(0, len(df_reviews), batch_size):
            batch_num += 1
            batch_df = df_reviews.iloc[i:i + batch_size].copy()

            # Train LDA model with num_topics
            lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

            # Get topic labels for each document in the batch
            topic_labels = get_topic_label(lda_model, corpus[i:i + batch_size], num_topics)
            topic_labels_list.extend(topic_labels)

            # Get topic summary (top words) for the current LDA model
            topic_summary = get_lda_topic_summary(lda_model, num_topics)

            # Add the topic labels as a new column in the batch dataframe
            batch_df[f'LDA_topic_{num_topics}'] = topic_labels
            batch_df[f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(batch_df)

            # Save the processed batch to a checkpoint file
            batch_df.to_csv(checkpoint_file, index=False, mode='a', header=not os.path.exists(checkpoint_file))
            print(f"Processed batch {batch_num} for {num_topics} topics and saved to checkpoint.")

        # Once all batches are processed, store the topic summaries
        all_topic_summaries[f'LDA_summary_{num_topics}_topics'] = topic_summary

    return df_reviews, all_topic_summaries

# Run LDA for topic numbers 3 to 6 and add topic labels and summaries to the dataframe
df_reviews_with_topics, lda_summaries = run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=2)

# Display the dataframe with the added topic columns and summaries
print(df_reviews_with_topics)

# Print the summaries for each topic number
for topic_num, summary in lda_summaries.items():
    print(f"{topic_num}:")
    for topic, keywords in summary.items():
        print(f"{topic}: {keywords}")
    print("\n")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
<ipython-input-7-001d90c1fc1d>:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews['processed_reviews'] = df_reviews['reviews'].apply(preprocess)


Processed batch 1 for 3 topics and saved to checkpoint.
Processed batch 2 for 3 topics and saved to checkpoint.
Processed batch 3 for 3 topics and saved to checkpoint.
Processed batch 4 for 3 topics and saved to checkpoint.
Processed batch 5 for 3 topics and saved to checkpoint.
Processed batch 6 for 3 topics and saved to checkpoint.
Processed batch 7 for 3 topics and saved to checkpoint.
Processed batch 8 for 3 topics and saved to checkpoint.
Processed batch 9 for 3 topics and saved to checkpoint.
Processed batch 10 for 3 topics and saved to checkpoint.
Processed batch 11 for 3 topics and saved to checkpoint.
Processed batch 12 for 3 topics and saved to checkpoint.
Processed batch 13 for 3 topics and saved to checkpoint.
Processed batch 14 for 3 topics and saved to checkpoint.
Processed batch 15 for 3 topics and saved to checkpoint.
Processed batch 16 for 3 topics and saved to checkpoint.
Processed batch 17 for 3 topics and saved to checkpoint.
Processed batch 18 for 3 topics and save

In [ ]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

# Download NLTK stopwords and punkt
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

# Sample DataFrame (replace this with your actual reviews)
data = {'reviews': ['The grading system was unclear.',
                    'The course material was outdated.',
                    'I enjoyed the lectures, but the workload was too high.',
                    'Assignments were good, but could use better feedback.',
                    'The instructor was excellent, but the exams were difficult.'],
        'course_id': ['python']*5}
df_reviews = pd.DataFrame(data)

# Preprocessing function
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Tokenizing and converting to lowercase
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Function to get top topics for each LDA model
def get_topic_label(lda_model, corpus, num_topics):
    # Get the most dominant topic for each review
    topic_labels = []
    for doc in corpus:
        topics = lda_model.get_document_topics(doc)
        dominant_topic = max(topics, key=lambda x: x[1])[0]  # Get the topic with the highest probability
        topic_labels.append(f"Topic {dominant_topic + 1}")
    return topic_labels

# Function to get LDA topic summaries (top words for each topic)
def get_lda_topic_summary(lda_model, num_topics, top_n=5):
    summary = {}
    for i in range(num_topics):
        # Get the top words for each topic
        words = lda_model.show_topic(i, topn=top_n)
        topic_keywords = " + ".join([f'{weight:.3f}*"{word}"' for word, weight in words])
        summary[f"Topic {i+1}"] = topic_keywords
    return summary

# Function to run LDA with different topic numbers and add them as columns to the dataframe
def run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=100, checkpoint_dir="checkpoints6"):
    all_topic_labels = {}
    all_topic_summaries = {}

    # Preprocess the reviews
    df_reviews['processed_reviews'] = df_reviews['reviews'].apply(preprocess)
    dictionary = corpora.Dictionary(df_reviews['processed_reviews'])
    corpus = [dictionary.doc2bow(review) for review in df_reviews['processed_reviews']]

    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Loop through different topic numbers (from topic_range)
    for num_topics in range(topic_range[0], topic_range[1] + 1):
        topic_labels_list = []

        # Check if checkpoint exists for this number of topics
        checkpoint_file = os.path.join(checkpoint_dir, f"lda_model_{num_topics}_topics.csv")

        if os.path.exists(checkpoint_file):
            print(f"Skipping LDA with {num_topics} topics, checkpoint found.")
            continue

        # Train LDA model with num_topics
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # Get topic labels for each review
        topic_labels = get_topic_label(lda_model, corpus, num_topics)
        all_topic_labels[f'LDA_topic_{num_topics}'] = topic_labels

        # Get topic summary (top words) for the current LDA model
        topic_summary = get_lda_topic_summary(lda_model, num_topics)
        all_topic_summaries[f'LDA_summary_{num_topics}_topics'] = topic_summary

        # Add the topic labels as a new column in the dataframe
        df_reviews[f'LDA_topic_{num_topics}'] = topic_labels

        # Store the topic summaries for later use
        df_reviews[f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)

        # Save the processed dataframe to a checkpoint file
        df_reviews.to_csv(checkpoint_file, index=False)
        print(f"Processed LDA with {num_topics} topics and saved to checkpoint.")

    return df_reviews, all_topic_summaries

# Run LDA for topic numbers 3 to 6 and add topic labels and summaries to the dataframe
df_reviews_with_topics, lda_summaries = run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=2)

# Display the dataframe with the added topic columns and summaries
print(df_reviews_with_topics)

# Print the summaries for each topic number
for topic_num, summary in lda_summaries.items():
    print(f"{topic_num}:")
    for topic, keywords in summary.items():
        print(f"{topic}: {keywords}")
    print("\n")


Processed LDA with 3 topics and saved to checkpoint.
Processed LDA with 4 topics and saved to checkpoint.
Processed LDA with 5 topics and saved to checkpoint.
Processed LDA with 6 topics and saved to checkpoint.
                                             reviews course_id  \
0                    The grading system was unclear.    python   
1                  The course material was outdated.    python   
2  I enjoyed the lectures, but the workload was t...    python   
3  Assignments were good, but could use better fe...    python   
4  The instructor was excellent, but the exams we...    python   

                                   processed_reviews LDA_topic_3  \
0                         [grading, system, unclear]     Topic 1   
1                       [course, material, outdated]     Topic 2   
2                [enjoyed, lectures, workload, high]     Topic 3   
3  [assignments, good, could, use, better, feedback]     Topic 1   
4          [instructor, excellent, exams, difficult

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# trying LDA with 3 - 6 topic clusters (different review topics)

In [ ]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

# Download NLTK stopwords and punkt
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

df_reviews = train_df.head(100)

# Preprocessing function
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Tokenizing and converting to lowercase
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Function to get top topics for each LDA model
def get_topic_label(lda_model, corpus, num_topics):
    # Get the most dominant topic for each review
    topic_labels = []
    for doc in corpus:
        topics = lda_model.get_document_topics(doc)
        dominant_topic = max(topics, key=lambda x: x[1])[0]  # Get the topic with the highest probability
        topic_labels.append(f"Topic {dominant_topic + 1}")
    return topic_labels

# Function to get LDA topic summaries (top words for each topic)
def get_lda_topic_summary(lda_model, num_topics, top_n=5):
    summary = {}
    for i in range(num_topics):
        # Get the top words for each topic
        words = lda_model.show_topic(i, topn=top_n)
        topic_keywords = " + ".join([f'{weight:.3f}*"{word}"' for word, weight in words])
        summary[f"Topic {i+1}"] = topic_keywords
    return summary

# Function to run LDA with different topic numbers and add them as columns to the dataframe
def run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=100, checkpoint_dir="checkpoints90"):
    all_topic_labels = {}
    all_topic_summaries = {}

    # Preprocess the reviews
    df_reviews['processed_reviews'] = df_reviews['reviews'].apply(preprocess)
    dictionary = corpora.Dictionary(df_reviews['processed_reviews'])
    corpus = [dictionary.doc2bow(review) for review in df_reviews['processed_reviews']]

    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Loop through different topic numbers (from topic_range)
    for num_topics in range(topic_range[0], topic_range[1] + 1):
        topic_labels_list = []

        # Check if checkpoint exists for this number of topics
        checkpoint_file = os.path.join(checkpoint_dir, f"lda_model_{num_topics}_topics.csv")

        if os.path.exists(checkpoint_file):
            print(f"Skipping LDA with {num_topics} topics, checkpoint found.")
            continue

        # Train LDA model with num_topics
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # Get topic labels for each review
        topic_labels = get_topic_label(lda_model, corpus, num_topics)
        all_topic_labels[f'LDA_topic_{num_topics}'] = topic_labels

        # Get topic summary (top words) for the current LDA model
        topic_summary = get_lda_topic_summary(lda_model, num_topics)
        all_topic_summaries[f'LDA_summary_{num_topics}_topics'] = topic_summary

        # Add the topic labels as a new column in the dataframe
        df_reviews[f'LDA_topic_{num_topics}'] = topic_labels

        # Store the topic summaries for later use
        df_reviews[f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)

        # Save the processed dataframe to a checkpoint file
        df_reviews.to_csv(checkpoint_file, index=False)
        print(f"Processed LDA with {num_topics} topics and saved to checkpoint.")

    return df_reviews, all_topic_summaries

# Run LDA for topic numbers 3 to 6 and add topic labels and summaries to the dataframe
df_reviews_with_topics, lda_summaries = run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=2)

# Display the dataframe with the added topic columns and summaries
print(df_reviews_with_topics)

# Print the summaries for each topic number
for topic_num, summary in lda_summaries.items():
    print(f"{topic_num}:")
    for topic, keywords in summary.items():
        print(f"{topic}: {keywords}")
    print("\n")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-15-06f4ff1b3579>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews['processed_reviews'] = df_reviews['reviews'].apply(preprocess)
<ipython-input-15-06f4ff1b3579>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews[f'LDA_topic_{num_topics}'] = topi

Processed LDA with 3 topics and saved to checkpoint.


<ipython-input-15-06f4ff1b3579>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews[f'LDA_topic_{num_topics}'] = topic_labels
<ipython-input-15-06f4ff1b3579>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews[f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)


Processed LDA with 4 topics and saved to checkpoint.


<ipython-input-15-06f4ff1b3579>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews[f'LDA_topic_{num_topics}'] = topic_labels
<ipython-input-15-06f4ff1b3579>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews[f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)


Processed LDA with 5 topics and saved to checkpoint.
Processed LDA with 6 topics and saved to checkpoint.
    Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0          46979         43342       52189   
1          15087         27963       33570   
2          32265          3290        4397   
3          44070         43900       52762   
4          18016         34850       42219   
..           ...           ...         ...   
95         29380         35428       42974   
96          6718         29502       35306   
97         39606         23341       28179   
98         43166         36371       44132   
99         49218         29542       35467   

                                              reviews date_reviews  rating  \
0   Andrew Ng has provided a fantastic resource fo...   2017-09-12       5   
1   Peer graded assingments take a lot of time bec...   2020-06-14       4   
2   This course was the first actual course for Pr...   2019-01-31       5   
3   The best course to start 

<ipython-input-15-06f4ff1b3579>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews[f'LDA_topic_{num_topics}'] = topic_labels
<ipython-input-15-06f4ff1b3579>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews[f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)


In [ ]:
import pandas as pd
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

# Download NLTK stopwords and punkt
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

df_reviews = train_df.head(100)

# Preprocessing function
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Tokenizing and converting to lowercase
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Function to get top topics for each LDA model
def get_topic_label(lda_model, corpus, num_topics):
    # Get the most dominant topic for each review
    topic_labels = []
    for doc in corpus:
        topics = lda_model.get_document_topics(doc)
        dominant_topic = max(topics, key=lambda x: x[1])[0]  # Get the topic with the highest probability
        topic_labels.append(f"Topic {dominant_topic + 1}")
    return topic_labels

# Function to get LDA topic summaries (top words for each topic)
def get_lda_topic_summary(lda_model, num_topics, top_n=5):
    summary = {}
    for i in range(num_topics):
        # Get the top words for each topic
        words = lda_model.show_topic(i, topn=top_n)
        topic_keywords = " + ".join([f'{weight:.3f}*"{word}"' for word, weight in words])
        summary[f"Topic {i+1}"] = topic_keywords
    return summary

# Function to run LDA with different topic numbers and add them as columns to the dataframe
def run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=100, checkpoint_dir="checkpoints189"):
    all_topic_labels = {}
    all_topic_summaries = {}

    # Preprocess the reviews
    df_reviews.loc[:, 'processed_reviews'] = df_reviews['reviews'].apply(preprocess)  # Use .loc here
    dictionary = corpora.Dictionary(df_reviews['processed_reviews'])
    corpus = [dictionary.doc2bow(review) for review in df_reviews['processed_reviews']]

    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Loop through different topic numbers (from topic_range)
    for num_topics in range(topic_range[0], topic_range[1] + 1):
        topic_labels_list = []

        # Check if checkpoint exists for this number of topics
        checkpoint_file = os.path.join(checkpoint_dir, f"lda_model_{num_topics}_topics.csv")

        if os.path.exists(checkpoint_file):
            print(f"Skipping LDA with {num_topics} topics, checkpoint found.")
            continue

        # Train LDA model with num_topics
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # Get topic labels for each review
        topic_labels = get_topic_label(lda_model, corpus, num_topics)
        all_topic_labels[f'LDA_topic_{num_topics}'] = topic_labels

        # Get topic summary (top words) for the current LDA model
        topic_summary = get_lda_topic_summary(lda_model, num_topics)
        all_topic_summaries[f'LDA_summary_{num_topics}_topics'] = topic_summary

        # Add the topic labels as a new column in the dataframe
        df_reviews.loc[:, f'LDA_topic_{num_topics}'] = topic_labels  # Use .loc here

        # Add the topic summary (as a string) to the dataframe
        df_reviews.loc[:, f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)  # Use .loc here

        # Save the processed dataframe to a checkpoint file
        df_reviews.to_csv(checkpoint_file, index=False)
        print(f"Processed LDA with {num_topics} topics and saved to checkpoint.")

    return df_reviews, all_topic_summaries

# Run LDA for topic numbers 3 to 6 and add topic labels and summaries to the dataframe
df_reviews_with_topics, lda_summaries = run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=2)

# Display the dataframe with the added topic columns and summaries
print(df_reviews_with_topics)

# Print the summaries for each topic number
for topic_num, summary in lda_summaries.items():
    print(f"{topic_num}:")
    for topic, keywords in summary.items():
        print(f"{topic}: {keywords}")
    print("\n")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
<ipython-input-20-b2a3c0631370>:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.loc[:, 'processed_reviews'] = df_reviews['reviews'].apply(preprocess)  # Use .loc here
<ipython-input-20-b2a3c0631370>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.loc[:, f'

Processed LDA with 3 topics and saved to checkpoint.


<ipython-input-20-b2a3c0631370>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.loc[:, f'LDA_topic_{num_topics}'] = topic_labels  # Use .loc here
<ipython-input-20-b2a3c0631370>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.loc[:, f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)  # Use .loc here


Processed LDA with 4 topics and saved to checkpoint.


<ipython-input-20-b2a3c0631370>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.loc[:, f'LDA_topic_{num_topics}'] = topic_labels  # Use .loc here
<ipython-input-20-b2a3c0631370>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.loc[:, f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)  # Use .loc here


Processed LDA with 5 topics and saved to checkpoint.
Processed LDA with 6 topics and saved to checkpoint.
    Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0          46979         43342       52189   
1          15087         27963       33570   
2          32265          3290        4397   
3          44070         43900       52762   
4          18016         34850       42219   
..           ...           ...         ...   
95         29380         35428       42974   
96          6718         29502       35306   
97         39606         23341       28179   
98         43166         36371       44132   
99         49218         29542       35467   

                                              reviews date_reviews  rating  \
0   Andrew Ng has provided a fantastic resource fo...   2017-09-12       5   
1   Peer graded assingments take a lot of time bec...   2020-06-14       4   
2   This course was the first actual course for Pr...   2019-01-31       5   
3   The best course to start 

<ipython-input-20-b2a3c0631370>:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.loc[:, f'LDA_topic_{num_topics}'] = topic_labels  # Use .loc here
<ipython-input-20-b2a3c0631370>:81: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.loc[:, f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)  # Use .loc here


**testing unsupervised labelling on a subset of the data**

In [ ]:
import pandas as pd
import torch
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

# Download NLTK stopwords and punkt
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

df_reviews = train_df.head(100)

# Preprocessing function
def preprocess(text):
    tokens = word_tokenize(text.lower())  # Tokenizing and converting to lowercase
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Function to get top topics for each LDA model
def get_topic_label(lda_model, corpus, num_topics):
    topic_labels = []
    for doc in corpus:
        topics = lda_model.get_document_topics(doc)
        dominant_topic = max(topics, key=lambda x: x[1])[0]  # Get the topic with the highest probability
        topic_labels.append(f"Topic {dominant_topic + 1}")
    return topic_labels

# Function to get LDA topic summaries (top words for each topic)
def get_lda_topic_summary(lda_model, num_topics, top_n=5):
    summary = {}
    for i in range(num_topics):
        words = lda_model.show_topic(i, topn=top_n)
        topic_keywords = " + ".join([f'{weight:.3f}*\"{word}\"' for word, weight in words])
        summary[f"Topic {i+1}"] = topic_keywords
    return summary

# Function to run LDA with different topic numbers and add them as columns to the dataframe
def run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), checkpoint_dir="/content/drive/MyDrive/checkpoints_test_unsupervised_pth"):
    all_topic_labels = {}
    all_topic_summaries = {}

    # Preprocess the reviews
    df_reviews = df_reviews.copy()  # Ensures modifications are on a copy of the original DataFrame
    df_reviews.loc[:, 'processed_reviews'] = df_reviews['reviews'].apply(preprocess)

    # Create dictionary and corpus for LDA
    dictionary = corpora.Dictionary(df_reviews['processed_reviews'])
    corpus = [dictionary.doc2bow(review) for review in df_reviews['processed_reviews']]

    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Loop through different topic numbers (from topic_range)
    for num_topics in range(topic_range[0], topic_range[1] + 1):
        checkpoint_file = os.path.join(checkpoint_dir, f"lda_model_{num_topics}_topics.pth")

        # Load checkpoint if it exists
        if os.path.exists(checkpoint_file):
            print(f"Loading LDA checkpoint for {num_topics} topics from {checkpoint_file}.")
            checkpoint = torch.load(checkpoint_file)
            all_topic_labels[f'LDA_topic_{num_topics}'] = checkpoint['topic_labels']
            all_topic_summaries[f'LDA_summary_{num_topics}_topics'] = checkpoint['topic_summary']
            continue

        # Train LDA model with num_topics
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # Get topic labels for each review
        topic_labels = get_topic_label(lda_model, corpus, num_topics)
        all_topic_labels[f'LDA_topic_{num_topics}'] = topic_labels

        # Get topic summary (top words) for the current LDA model
        topic_summary = get_lda_topic_summary(lda_model, num_topics)
        all_topic_summaries[f'LDA_summary_{num_topics}_topics'] = topic_summary

        # Assign the topic labels and summary to the DataFrame using .loc
        df_reviews.loc[:, f'LDA_topic_{num_topics}'] = topic_labels
        df_reviews.loc[:, f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)

        # Save checkpoint
        torch.save({
            'topic_labels': topic_labels,
            'topic_summary': topic_summary
        }, checkpoint_file)

        print(f"Processed LDA with {num_topics} topics and saved checkpoint to {checkpoint_file}.")

    return df_reviews, all_topic_summaries

# Run LDA for topic numbers 3 to 6 and add topic labels and summaries to the dataframe
df_reviews_with_topics, lda_summaries = run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6))

# Display the dataframe with the added topic columns and summaries
print(df_reviews_with_topics)

# Print the summaries for each topic number
for topic_num, summary in lda_summaries.items():
    print(f"{topic_num}:")
    for topic, keywords in summary.items():
        print(f"{topic}: {keywords}")
    print("\n")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processed LDA with 3 topics and saved checkpoint to /content/drive/MyDrive/checkpoints_test_unsupervised_pth/lda_model_3_topics.pth.
Processed LDA with 4 topics and saved checkpoint to /content/drive/MyDrive/checkpoints_test_unsupervised_pth/lda_model_4_topics.pth.
Processed LDA with 5 topics and saved checkpoint to /content/drive/MyDrive/checkpoints_test_unsupervised_pth/lda_model_5_topics.pth.
Processed LDA with 6 topics and saved checkpoint to /content/drive/MyDrive/checkpoints_test_unsupervised_pth/lda_model_6_topics.pth.
    Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0          46979         43342       52189   
1          15087         27963       33570   
2          32265          3290        4397   
3          44070         43900       52762   
4          18016         34850       42219   
..           ...           ...         ...   
95         29380         35428       42974   
96          6718         29502       35306   
97         39606         23341       28179   
98      

**unsupervised labelling for all the reviews**

In [ ]:
import pandas as pd
import torch
from gensim import corpora
from gensim.models import LdaModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os

# Download NLTK stopwords and punkt
nltk.download('stopwords')
nltk.download('punkt')
stop_words = set(stopwords.words('english'))

df_reviews = train_df  # Replace with your actual dataframe

# Preprocessing function with a check for NaN or non-string values
def preprocess(text):
    if not isinstance(text, str):  # Check if the text is a string
        return []  # Return an empty list if the text is not a string or is NaN
    tokens = word_tokenize(text.lower())  # Tokenizing and converting to lowercase
    tokens = [word for word in tokens if word.isalpha() and word not in stop_words]
    return tokens

# Function to get top topics for each LDA model
def get_topic_label(lda_model, corpus, num_topics):
    topic_labels = []
    for doc in corpus:
        topics = lda_model.get_document_topics(doc)
        dominant_topic = max(topics, key=lambda x: x[1])[0]  # Get the topic with the highest probability
        topic_labels.append(f"Topic {dominant_topic + 1}")
    return topic_labels

# Function to get LDA topic summaries (top words for each topic)
def get_lda_topic_summary(lda_model, num_topics, top_n=5):
    summary = {}
    for i in range(num_topics):
        words = lda_model.show_topic(i, topn=top_n)
        topic_keywords = " + ".join([f'{weight:.3f}*\"{word}\"' for word, weight in words])
        summary[f"Topic {i+1}"] = topic_keywords
    return summary

# Function to run LDA with different topic numbers and add them as columns to the dataframe
def run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=100, checkpoint_dir="/content/drive/MyDrive/checkpoints_boo"):
    all_topic_labels = {}
    all_topic_summaries = {}

    # Preprocess the reviews
    df_reviews = df_reviews.copy()  # Ensures modifications are on a copy of the original DataFrame
    df_reviews['reviews'] = df_reviews['reviews'].fillna('').astype(str)  # Handle missing reviews
    df_reviews['processed_reviews'] = df_reviews['reviews'].apply(preprocess)

    # Create dictionary and corpus for LDA
    dictionary = corpora.Dictionary(df_reviews['processed_reviews'])
    corpus = [dictionary.doc2bow(review) for review in df_reviews['processed_reviews']]

    # Create checkpoint directory if it doesn't exist
    os.makedirs(checkpoint_dir, exist_ok=True)

    # Loop through different topic numbers (from topic_range)
    for num_topics in range(topic_range[0], topic_range[1] + 1):
        checkpoint_file = os.path.join(checkpoint_dir, f"lda_model_{num_topics}_topics.pth")

        # Load checkpoint if it exists
        if os.path.exists(checkpoint_file):
            print(f"Loading LDA checkpoint for {num_topics} topics from {checkpoint_file}.")
            checkpoint = torch.load(checkpoint_file)
            all_topic_labels[f'LDA_topic_{num_topics}'] = checkpoint['topic_labels']
            all_topic_summaries[f'LDA_summary_{num_topics}_topics'] = checkpoint['topic_summary']
            continue

        # Train LDA model with num_topics
        lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)

        # Get topic labels for each review
        topic_labels = get_topic_label(lda_model, corpus, num_topics)
        all_topic_labels[f'LDA_topic_{num_topics}'] = topic_labels

        # Get topic summary (top words) for the current LDA model
        topic_summary = get_lda_topic_summary(lda_model, num_topics)
        all_topic_summaries[f'LDA_summary_{num_topics}_topics'] = topic_summary

        # Assign the topic labels and summary to the DataFrame using .loc
        df_reviews[f'LDA_topic_{num_topics}'] = topic_labels
        df_reviews[f'LDA_summary_{num_topics}_topics'] = [topic_summary] * len(df_reviews)

        # Save checkpoint
        torch.save({
            'topic_labels': topic_labels,
            'topic_summary': topic_summary
        }, checkpoint_file)

        print(f"Processed LDA with {num_topics} topics and saved checkpoint to {checkpoint_file}.")

    return df_reviews, all_topic_summaries

# Run LDA for topic numbers 3 to 6 and add topic labels and summaries to the dataframe
df_reviews_with_topics, lda_summaries = run_lda_with_labels_and_summaries(df_reviews, topic_range=(3, 6), batch_size=100)

# Display the dataframe with the added topic columns and summaries
print(df_reviews_with_topics)

# Print the summaries for each topic number
for topic_num, summary in lda_summaries.items():
    print(f"{topic_num}:")
    for topic, keywords in summary.items():
        print(f"{topic}: {keywords}")
    print("\n")


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Processed LDA with 3 topics and saved checkpoint to /content/drive/MyDrive/checkpoints_boo/lda_model_3_topics.pth.
Processed LDA with 4 topics and saved checkpoint to /content/drive/MyDrive/checkpoints_boo/lda_model_4_topics.pth.
Processed LDA with 5 topics and saved checkpoint to /content/drive/MyDrive/checkpoints_boo/lda_model_5_topics.pth.
Processed LDA with 6 topics and saved checkpoint to /content/drive/MyDrive/checkpoints_boo/lda_model_6_topics.pth.
       Unnamed: 0.2  Unnamed: 0.1  Unnamed: 0  \
0           46979.0       43342.0     52189.0   
1           15087.0       27963.0     33570.0   
2           32265.0        3290.0      4397.0   
3           44070.0       43900.0     52762.0   
4           18016.0       34850.0     42219.0   
...             ...           ...         ...   
39733       14444.0       32000.0     38501.0   
39734       46433.0       25044.0     30104.0   
39735       22140.0       27877.0     33439.0   
39736       29009.0        2544.0      3541.0   
3

In [ ]:
df_reviews_with_topics

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,reviews,date_reviews,rating,course_id,month,departments,name,...,sentiment_label_from_review,processed_reviews,LDA_topic_3,LDA_summary_3_topics,LDA_topic_4,LDA_summary_4_topics,LDA_topic_5,LDA_summary_5_topics,LDA_topic_6,LDA_summary_6_topics
0,46979.0,43342.0,52189.0,Andrew Ng has provided a fantastic resource fo...,2017-09-12,5.0,machine-learning,2017-09,Computer Science,Machine Learning,...,positive,"[andrew, ng, provided, fantastic, resource, lo...",Topic 3,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 4,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 2,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 2,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
1,15087.0,27963.0,33570.0,Peer graded assingments take a lot of time bec...,2020-06-14,4.0,python-databases,2020-06,Computer Science,Using Databases with Python,...,positive,"[peer, graded, assingments, take, lot, time, n...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 2,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 1,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 3,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
2,32265.0,3290.0,4397.0,This course was the first actual course for Pr...,2019-01-31,5.0,python,2019-01,Computer Science,Programming for Everybody (Getting Started wit...,...,positive,"[course, first, actual, course, programming, c...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 1,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 3,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 2,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
3,44070.0,43900.0,52762.0,The best course to start with if you are enter...,2017-08-20,5.0,machine-learning,2017-08,Computer Science,Machine Learning,...,positive,"[best, course, start, entering, field, machine...",Topic 3,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 4,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 2,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 2,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
4,18016.0,34850.0,42219.0,I wanted to learn more about Functional Progra...,2020-07-12,4.0,programming-languages,2020-07,Computer Science,"Programming Languages, Part A",...,positive,"[wanted, learn, functional, programming, prepa...",Topic 1,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 1,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 3,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 4,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39733,14444.0,32000.0,38501.0,This course is one of the best courses that sh...,2019-02-12,4.0,python-data-visualization,2019-02,Computer Science,"Capstone: Retrieving, Processing, and Visualiz...",...,positive,"[course, one, best, courses, show, python, abi...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 4,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 5,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 4,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
39734,46433.0,25044.0,30104.0,"The details about Objects, References and Alia...",2019-08-08,4.0,python-basics,2019-08,Computer Science,Python Basics,...,positive,"[details, objects, references, aliases, clear,...",Topic 1,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 3,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 1,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 6,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
39735,22140.0,27877.0,33439.0,This course gives very basic information about...,2016-02-19,4.0,python-databases,2016-02,Computer Science,Using Databases with Python,...,positive,"[course, gives, basic, information, using, pyt...",Topic 1,"{'

In [ ]:
df_reviews_with_topics.drop(columns=["Unnamed: 0", "Unnamed: 0.1", "Unnamed: 0.2"], axis=1, inplace=True)


In [ ]:
df_reviews_with_topics

,reviews,date_reviews,rating,course_id,month,departments,name,institution,sentiment_from_rating,emotion_label,...,sentiment_label_from_review,processed_reviews,LDA_topic_3,LDA_summary_3_topics,LDA_topic_4,LDA_summary_4_topics,LDA_topic_5,LDA_summary_5_topics,LDA_topic_6,LDA_summary_6_topics
0,Andrew Ng has provided a fantastic resource fo...,2017-09-12,5.0,machine-learning,2017-09,Computer Science,Machine Learning,Stanford University,positive,admiration,...,positive,"[andrew, ng, provided, fantastic, resource, lo...",Topic 3,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 4,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 2,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 2,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
1,Peer graded assingments take a lot of time bec...,2020-06-14,4.0,python-databases,2020-06,Computer Science,Using Databases with Python,University of Michigan,positive,neutral,...,positive,"[peer, graded, assingments, take, lot, time, n...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 2,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 1,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 3,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
2,This course was the first actual course for Pr...,2019-01-31,5.0,python,2019-01,Computer Science,Programming for Everybody (Getting Started wit...,University of Michigan,positive,admiration,...,positive,"[course, first, actual, course, programming, c...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 1,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 3,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 2,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
3,The best course to start with if you are enter...,2017-08-20,5.0,machine-learning,2017-08,Computer Science,Machine Learning,Stanford University,positive,approval,...,positive,"[best, course, start, entering, field, machine...",Topic 3,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 4,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 2,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 2,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
4,I wanted to learn more about Functional Progra...,2020-07-12,4.0,programming-languages,2020-07,Computer Science,"Programming Languages, Part A",University of Washington,positive,approval,...,positive,"[wanted, learn, functional, programming, prepa...",Topic 1,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 1,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 3,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 4,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39733,This course is one of the best courses that sh...,2019-02-12,4.0,python-data-visualization,2019-02,Computer Science,"Capstone: Retrieving, Processing, and Visualiz...",University of Michigan,positive,admiration,...,positive,"[course, one, best, courses, show, python, abi...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 4,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 5,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 4,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
39734,"The details about Objects, References and Alia...",2019-08-08,4.0,python-basics,2019-08,Computer Science,Python Basics,University of Michigan,positive,admiration,...,positive,"[details, objects, references, aliases, clear,...",Topic 1,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 3,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 1,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 6,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
39735,This course gives very basic information about...,2016-02-19,4.0,python-databases,2016-02,Computer S

In [ ]:
df_reviews_with_topics.to_csv("/content/drive/MyDrive/df_reviews_with_topics_unsupervised.csv")

In [ ]:
import pandas as pd
df_topics_super_labels = pd.read_csv(("/content/drive/MyDrive/df_reviews_with_topics_unsupervised.csv"))
df_topics_super_labels.drop(columns=["Unnamed: 0"], axis=1, inplace=True)
df_topics_super_labels[["reviews", "sentiment_label_from_review", "processed_reviews",	"LDA_topic_3", 	"LDA_summary_3_topics", 	"LDA_topic_4",	"LDA_summary_4_topics", "LDA_topic_5", "LDA_summary_5_topics", "LDA_topic_6",	"LDA_summary_6_topics"]].iloc[:100]

,reviews,sentiment_label_from_review,processed_reviews,LDA_topic_3,LDA_summary_3_topics,LDA_topic_4,LDA_summary_4_topics,LDA_topic_5,LDA_summary_5_topics,LDA_topic_6,LDA_summary_6_topics
0,Andrew Ng has provided a fantastic resource fo...,positive,"['andrew', 'ng', 'provided', 'fantastic', 'res...",Topic 3,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 4,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 2,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 2,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
1,Peer graded assingments take a lot of time bec...,positive,"['peer', 'graded', 'assingments', 'take', 'lot...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 2,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 1,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 3,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
2,This course was the first actual course for Pr...,positive,"['course', 'first', 'actual', 'course', 'progr...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 1,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 3,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 2,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
3,The best course to start with if you are enter...,positive,"['best', 'course', 'start', 'entering', 'field...",Topic 3,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 4,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 2,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 2,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
4,I wanted to learn more about Functional Progra...,positive,"['wanted', 'learn', 'functional', 'programming...",Topic 1,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 1,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 3,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 4,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
...,...,...,...,...,...,...,...,...,...,...,...
95,The Course cleared all of my doubts regarding ...,positive,"['course', 'cleared', 'doubts', 'regarding', '...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 2,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 1,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 6,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
96,"Very unsatisfied. At the end, I feel I learnt ...",negative,"['unsatisfied', 'end', 'feel', 'learnt', 'expl...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 2,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 1,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 6,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
97,"A very clear, well structured course for learn...",positive,"['clear', 'well', 'structured', 'course', 'lea...",Topic 3,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 3,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 2,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 1,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."
98,"Great course, though was a bit difficult for m...",positive,"['great', 'course', 'though', 'bit', 'difficul...",Topic 2,"{'Topic 1': '0.089*""course"" + 0.055*""python"" +...",Topic 2,"{'Topic 1': '0.088*""course"" + 0.046*""python"" +...",Topic 1,"{'Topic 1': '0.022*""course"" + 0.022*""assignmen...",Topic 3,"{'Topic 1': '0.083*""course"" + 0.034*""easy"" + 0..."


Un-supervised lablling notes:

3 topics -> too big of categories
1: course material
2: Utility/Usefulness
3 : Learning outcomes

4 topics ->
4 = Learning Outcomes
2 = course structure and assignements
1 = Learning/Understanding

5 topics is deffo the most comprehensive:
1: course material
2: learning outcomes
3: course structure and organisation
4: Utility / usefulness
5: Course Engagement and Course Feelings(Sentiment)

6 are a bit confused and have too much noise, so focused on manual inspection results from 3 and 4.

Assigning Topic Names using the topic summary for each cluster

In [ ]:
df_topics_super_labels.drop(columns=["LDA_summary_3_topics", "LDA_summary_4_topics", "LDA_summary_5_topics", "LDA_summary_6_topics", "LDA_topic_6"], axis=1, inplace=True)
df_topics_super_labels[["reviews", "sentiment_label_from_review", "LDA_topic_3", "LDA_topic_4", "LDA_topic_5"]].iloc[:100]

,reviews,sentiment_label_from_review,LDA_topic_3,LDA_topic_4,LDA_topic_5
0,Andrew Ng has provided a fantastic resource fo...,positive,Topic 3,Topic 4,Topic 2
1,Peer graded assingments take a lot of time bec...,positive,Topic 2,Topic 2,Topic 1
2,This course was the first actual course for Pr...,positive,Topic 2,Topic 1,Topic 3
3,The best course to start with if you are enter...,positive,Topic 3,Topic 4,Topic 2
4,I wanted to learn more about Functional Progra...,positive,Topic 1,Topic 1,Topic 3
...,...,...,...,...,...
95,The Course cleared all of my doubts regarding ...,positive,Topic 2,Topic 2,Topic 1
96,"Very unsatisfied. At the end, I feel I learnt ...",negative,Topic 2,Topic 2,Topic 1
97,"A very clear, well structured course for learn...",positive,Topic 3,Topic 3,Topic 2
98,"Great course, though was a bit difficult for m...",positive,Topic 2,Topic 2,Topic 1


In [ ]:
# developing consistent labels across the the different iterations of unsupervised modelling
df_topics_super_labels['LDA_topic_3'] = df_topics_super_labels['LDA_topic_3'].replace({
    'Topic 1': 'Course Material and Understanding',
    'Topic 2': 'Usefulness',
    'Topic 3': 'Learning Outcomes'
})


df_topics_super_labels['LDA_topic_4'] = df_topics_super_labels['LDA_topic_4'].replace({
    'Topic 1': 'Course Material and Understanding',
    'Topic 2': 'Course Structure',
    'Topic 3': 'Course Engagement and Course Feelings(Sentiment)',
    'Topic 4': 'Learning Outcomes and Utility'

})

df_topics_super_labels['LDA_topic_5'] = df_topics_super_labels['LDA_topic_5'].replace({
    'Topic 1': 'Course Material',
    'Topic 2': 'Learning Outcomes and Utility',
    'Topic 3': 'Course Structure and Organisation',
    'Topic 4': 'Utility and Usefulness',
    'Topic 5': 'Course Engagement and Course Feelings(Sentiment)'
})

df_topics_super_labels[["reviews", "sentiment_label_from_review", "LDA_topic_3", "LDA_topic_4", "LDA_topic_5"]].iloc[:100]

,reviews,sentiment_label_from_review,LDA_topic_3,LDA_topic_4,LDA_topic_5
0,Andrew Ng has provided a fantastic resource fo...,positive,Learning Outcomes,Learning Outcomes,Learning Outcomes
1,Peer graded assingments take a lot of time bec...,positive,Utility and Usefulness,Course Structure,Course Material
2,This course was the first actual course for Pr...,positive,Utility and Usefulness,Course Material and Understanding,Course Structure and Organisation
3,The best course to start with if you are enter...,positive,Learning Outcomes,Learning Outcomes,Learning Outcomes
4,I wanted to learn more about Functional Progra...,positive,Course Material,Course Material and Understanding,Course Structure and Organisation
...,...,...,...,...,...
95,The Course cleared all of my doubts regarding ...,positive,Utility and Usefulness,Course Structure,Course Material
96,"Very unsatisfied. At the end, I feel I learnt ...",negative,Utility and Usefulness,Course Structure,Course Material
97,"A very clear, well structured course for learn...",positive,Learning Outcomes,Course Engagement and Course Feelings(Sentiment),Learning Outcomes
98,"Great course, though was a bit difficult for m...",positive,Utility and Usefulness,Course Structure,Course Material


# Performance Notes for Unsupervised Topic Modelling:

3 Clusters:
* 83%

4 Clusters:
* 86%


5 Clusters:
* 77%

Will likely go with just 4 Clusters, maybe 3 and 4 Clusters and combine the results with the automatic labelling.

In [ ]:
df_topics_super_labels.to_csv("/content/drive/MyDrive/df_topics_unsupervised_labels_updated.csv")